In [1]:
import pandas as pd 
import numpy as np
import math

# DT 문제 1번

#### ※함수에 들어가 있는 변수나 flow는 본인이 바꾸셔도 가능하며 결과만 똑같이 나오면 됩니다!
#### ※hard코딩(이 데이터셋에만 적용되는 코딩방법) 말고 전체 데이터에 적용 가능하게 함수를 짜주셔야 합니다.

# Data Loading

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


# 1. Gini 계수에 대한 함수 만들기.

- Input은 Dataframe과 label이름으로 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다!

In [3]:
rows=len(pd_data)
rows

14

In [4]:
list(pd_data['class_buys_computer']).count('yes')

9

In [5]:
def get_gini(df, label):
    rows=len(df)
    yes=list(df[label]).count('yes')
    en=yes/rows
    #1-p2^2-p3^2
    gini=1-en**2-(1-en)**2
    return gini

In [6]:
get_gini(pd_data, 'class_buys_computer')

0.4591836734693877

# 2. Feature의 Class를 이진 분류로 만들기
## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})
- Powerset은 모든 조합을 뱉어내게 됩니다. 이건 그냥 완성된걸 드릴게요.

In [7]:
from itertools import chain, combinations

def powerset(feature_class):
    listed_data = list(feature_class)
    chain_set = chain.from_iterable(combinations(listed_data, i) 
                                    for i in range(len(listed_data)+1))
    return [set_data for set_data in chain_set]

In [8]:
powerset(pd_data.age.unique())

[(),
 ('youth',),
 ('middle_aged',),
 ('senior',),
 ('youth', 'middle_aged'),
 ('youth', 'senior'),
 ('middle_aged', 'senior'),
 ('youth', 'middle_aged', 'senior')]

In [9]:
type(powerset(pd_data.age.unique()))

list

- 저 위에 부분 집합 중 우리가 원하는 집합(이진 분류)만 골라 내야하겠죠?
- 그 함수를 get_binary_split로 완성해주세요!
- 완성된 내용은 다음과 같이 나와야 합니다.

In [10]:
a=[1,2,3,4]

In [11]:
a[1:len(a)]

[2, 3, 4]

In [12]:
def get_binary_split(df, attribute):
    fullset=powerset(df[attribute].unique())
    result=fullset[1:len(fullset)-1]
    result = [list(i) for i in result]
    return result

In [13]:

list(get_binary_split(pd_data, "age"))

[['youth'],
 ['middle_aged'],
 ['senior'],
 ['youth', 'middle_aged'],
 ['youth', 'senior'],
 ['middle_aged', 'senior']]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 함수를 사용하여 만들어주세요!
- DataFrame의 index를 사용하여 만들면 굉장히 편합니다..! 예시를 아래에서 보여드릴게요.
- 결과는 아래 아래 아래 줄과 같아야 합니다.

In [14]:
A = set([1,2,3])
B = set([3,4,5])
print(A.union(B))
#A와 B의 합집합
#0~6개의 행을 뽑아 옴. 
pd_data.loc[A.union(B)]

{1, 2, 3, 4, 5}


,age,income,student,credit_rating,class_buys_computer
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no


In [119]:
#list(pd_data.loc[list(pd_data[(pd_data.age=='youth')].index)]['class_buys_computer']).count('yes')
len(pd_data)

14

In [ ]:
"""def get_gini(df, label):
    rows=len(df)
    yes=list(df[label]).count('yes')
    en=yes/rows
    #1-p2^2-p3^2
    gini=1-en**2-(1-en)**2
    return gini
    """

In [15]:
#데이터,x,y
def get_attribute_gini_index(df, attribute, label):
    #attribute의 부분집합에서 필요한 집합만을 뽑고
    split_list=list(get_binary_split(df, attribute))
    result = {}
    #길이가 하나인 경우만 계산을 하고 나머지는 여집합으로 돌림.
    for i in split_list:
        if(len(i)==1):
#             print(i)
            index_i=list(df[(df[attribute]==i[0])].index)
#             print(index_i)
            index_e=list(df[(df[attribute]!=i[0])].index)
            df_i=df.loc[index_i]
            df_e=df.loc[index_e]
            #print(i)
            #print(df_i)
            #print(df_e)
            p1=(len(df_i)/len(pd_data))*get_gini(df_i,'class_buys_computer')
            p2=(len(df_e)/len(pd_data))*get_gini(df_e,'class_buys_computer')
            result[i[0]]=p1+p2
        else:
            pass
        
    return result

In [16]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{'youth': 0.3936507936507937,
 'middle_aged': 0.35714285714285715,
 'senior': 0.45714285714285713}

- 여기서 가장 작은값으로 분류를 해야겠죠?

In [17]:
min(get_attribute_gini_index(pd_data, "age", "class_buys_computer").items())

('middle_aged', 0.35714285714285715)

# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) income의 이진분류를 얻는 함수 get_binary_split(pd_data, "income")을 통해 보여주세요.

## 문제2) 가장 Gini계수가 !낮!은 Feature 즉 분류를 하는데 가장 중요한 변수를 선정하시고 get_attribute_gini_index함수를 통해 Gini index를 제시해주세요.

## 문제3) 2에서 구한 Feature로 DataFrame을 분류 해주시고 나눠진 2개의 클래스에서 각각 다음으로 중요한 Feature를 선정해주시고 Gini index를 제시해주세요.

## - 문제 1) 

In [42]:
get_binary_split(pd_data, "income")

[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

## - 문제2)

In [40]:

#모든 칼럼 이진분류 조사해 frist split feature 선정

columns=list(pd_data.columns)
min_feature=[]


for i in columns:
    print(i,"의 이진분류의 gini 최소값",min(get_attribute_gini_index(pd_data,i, "class_buys_computer").items()))
    


age 의 이진분류의 gini 최소값 ('middle_aged', 0.35714285714285715)
income 의 이진분류의 gini 최소값 ('high', 0.44285714285714284)
student 의 이진분류의 gini 최소값 ('no', 0.3673469387755103)
credit_rating 의 이진분류의 gini 최소값 ('excellent', 0.42857142857142855)
class_buys_computer 의 이진분류의 gini 최소값 ('no', 0.0)


## 결과) gini 계수를 이용한 CART 알고리즘
frist feature는 age,Binary split은 middle_aged/ senior,youth 로 한다.

## - 문제 3

In [48]:
index_i=list(pd_data[(pd_data["age"]=='middle_aged')].index)
print(index_i)
index_e=list(pd_data[(pd_data["age"]!='middle_aged')].index)
#midlle_aged/ youth, senior 로 데이터 분할

df_m=pd_data.loc[index_i]
df_em=pd_data.loc[index_e]

[2, 6, 11, 12]


### second split feature 구하기

- df_m ('age'- middle aged)

In [60]:
#frist split feature 였던 age delete
#del df_m['age']
df_m
# df_m 데이터는 모든 feature에서 class_buys_computer값이 'yes'임으로 더이상 가지 치기를 하지 않아도 된다. (모든 경우에서 확률이 1이기 때문)

,income,student,credit_rating,class_buys_computer
2,high,no,fair,yes
6,low,yes,excellent,yes
11,medium,no,excellent,yes
12,high,yes,fair,yes


- df_em ('age'-senior, youth)

In [62]:
#middle_aged data
del df_em['age']

In [64]:
columns=list(df_em.columns)
for i in columns:
    print(i,"의 이진분류의 gini 최소값",min(get_attribute_gini_index(df_em,i, "class_buys_computer").items()))

income 의 이진분류의 gini 최소값 ('high', 0.26785714285714285)
student 의 이진분류의 gini 최소값 ('no', 0.22857142857142848)
credit_rating 의 이진분류의 gini 최소값 ('excellent', 0.2976190476190476)
class_buys_computer 의 이진분류의 gini 최소값 ('no', 0.0)


In [68]:
df_em['student'].unique()

array(['no', 'yes'], dtype=object)

### 결과) student의 gini 계수가 가장 작음으로, df_em 데이터의 두번째 split feature는 student가 됨이 바람직 하다.